 ## WILL NOT WORK LOCALLY, REQUIRES SAGE TO RUN

In [3]:
from IPython.display import clear_output

In [4]:
def random_curve_and_point(bits=40):
    """
    Returns:
      p, a, b, P, n
    where:
      - p is a prime (~bits bits)
      - curve is y^2 = x^3 + a x + b over F_p
      - P is a random point
      - n is the order of P
    """
    while True:
        p = random_prime(2^bits, lbound=2^(bits-1))
        F = GF(p)

        # pick non-singular curve: 4a^3 + 27b^2 != 0
        while True:
            a = F.random_element()
            b = F.random_element()
            if 4*a^3 + 27*b^2 != 0:
                break

        E = EllipticCurve(F, [a, b])

        P = E.random_point()
        if P == E(0):        # point at infinity, useless
            # drop big objects before next iteration
            del E, F, P
            continue

        n = P.order()
        if n == 0 or n == 1:
            del E, F, P, n
            continue

        # at this point we keep only what we need
        return p, a, b, P, n

In [5]:
import re
def parse_factor_sage(fac):
    """
    Convert Sage factor(n) into Python list of (prime, exponent).
    Accepts either a Sage IntegerFactorization or a string like '31 * 68819 * 440269'.
    """
    # Case 1: It's already a Sage factorization object
    if hasattr(fac, "__iter__") and not isinstance(fac, str):
        try:
            return [(int(p), int(e)) for (p, e) in fac]
        except Exception:
            pass  # fall through to string handling
    
    # Case 2: Fall back to string parsing
    s = str(fac)
    parts = [p.strip() for p in s.split("*")]
    out = []
    for p in parts:
        if "^" in p:
            base, exp = p.split("^")
            out.append((int(base), int(exp)))
        else:
            out.append((int(p), 1))
    return out

In [6]:
def get_prime_order_curve(bits=40):
    while True:
        p, a, b, P, n = random_curve_and_point(bits)
        fac = factor(n)
        if n.is_prime():
            print("Found prime-order point!")
            print("order n has", n.nbits(), "bits")
            print("factor(n) =", fac)

            x, y = P.xy()
            print("\nCopy these into Python:\n")
            print("p  =", int(p))
            print("a  =", int(a))
            print("b  =", int(b))
            print("Gx =", int(x))
            print("Gy =", int(y))
            print("n  =", int(n))
            return p, a, b, P, n

p, a, b, P, n = get_prime_order_curve(bits=40)

Found prime-order point!
order n has 40 bits
factor(n) = 769812911597

Copy these into Python:

p  = 769812538459
a  = 352509587180
b  = 630366912856
Gx = 353969310423
Gy = 296185186781
n  = 769812911597


In [7]:
def get_composite_order_curve(bits=40):
    while True:
        p, a, b, P, n = random_curve_and_point(bits)

        # skip prime-order points early, no factor() call
        if n.is_prime():
            del P, n
            continue

        fac = factor(n)

        # we only handle square-free n in this PH implementation
        #if not all(e == 1 for (_, e) in fac):
            #del P, n, fac
            #continue

        # optional: prefer curves where the largest factor isn't too big
        max_factor = max(int(q) for (q, e) in fac)

        clear_output(wait=True)
        print("Trying n with factors:", fac)

        # tweak condition if you want smaller/larger factors
        if max_factor <= 2^(20):
            print("\nFound composite-order point!")
            print("order n has", n.nbits(), "bits")
            print("factor(n) =", fac)

            x, y = P.xy()
            print("\nCopy these into Python:\n")
            print("p_c  =", int(p))
            print("a_c  =", int(a))
            print("b_c  =", int(b))
            print("Gc_x =", int(x))
            print("Gc_y =", int(y))
            print("n_c  =", int(n))
            print("factor_n_c =", parse_factor_sage(fac))

            # return only what you actually need downstream
            return p, a, b, P, n, fac

        # if we didn't accept this curve, drop large objects before looping
        del P, n, fac

In [11]:
 p_c, a_c, b_c, P_c, n_c, fac_c = get_composite_order_curve(bits=40)

Trying n with factors: 3 * 5 * 1613 * 5651 * 7577

Found composite-order point!
order n has 40 bits
factor(n) = 3 * 5 * 1613 * 5651 * 7577

Copy these into Python:

p_c  = 1035972675893
a_c  = 206712200754
b_c  = 139762077084
Gc_x = 81155312250
Gc_y = 447657618032
n_c  = 1035972485265
factor_n_c = [(3, 1), (5, 1), (1613, 1), (5651, 1), (7577, 1)]


In [9]:
p_c, a_c, b_c, P_c, n_c, fac_c = get_composite_order_curve(bits=128)

Trying n with factors: 2^2 * 7 * 11 * 239 * 101399 * 560159 * 380188261 * 200425467544339


KeyboardInterrupt: 